In [1]:
%matplotlib inline

import gensim
import logging
import math
import nltk
import os
import pyLDAvis.gensim_models
import random
import re
import spacy
import json
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import cm

from imageio import imread
from wordcloud import WordCloud

from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
from nltk.corpus import stopwords # est-ce que c'est le même de sklearn ?
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures
from nltk.tokenize import RegexpTokenizer
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings("ignore")

C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: Depr

C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\keras\utils\image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\keras\utils\image_utils.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\keras\utils\image_utils.py:39: DeprecationWarning: HAMMING is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.HAMMING instead.
  'hamming': pil_image.HAMMING,
C:\Users\brune\anacon

In [2]:
with open(os.path.join('posts_maior_propre.json'), 'r', encoding = 'UTF-8') as fin_M2 :
    data = json.load(fin_M2)

In [3]:
index = ''
a = ''
list_clean_content = []

for j in range(len(data)) :
    index = data[j]
    a = index.replace( "\n", "")
    list_clean_content.append(a)

In [4]:
with open("Stop-words-french.txt", "r", encoding='utf8') as f:
    stop_words = [line.rstrip("\n") for line in f.readlines()]

In [5]:
def sent_to_words(sentences) :
    for sentence in sentences :
        yield(simple_preprocess(str(sentence), deacc = True))

In [6]:
doc_set = list_clean_content
data_words = list(sent_to_words(doc_set))

In [7]:
ndocs = len(data_words)
print(ndocs)

110895


In [8]:
stop_words = stop_words
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words_nostops = remove_stopwords(data_words)

In [10]:
# création du dictionnaire
dico = corpora.Dictionary(data_words_nostops)

# ce qui permet par ex. de filtrer le vocabulaire
dico.filter_extremes(no_below = 5, no_above = 0.5)

# Create Corpus
texts = data_words_nostops

# matrice Term Document Frequency
corpus = [dico.doc2bow(text) for text in texts]

In [10]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=1, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        lda_model_coh = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dico, per_word_topics=True,update_every=1,
                                           chunksize=100,random_state=80,
                                           passes=10,
                                           alpha='auto')
        model_list.append(lda_model_coh)
        coherence_model_lda = CoherenceModel(model=lda_model_coh, texts=data_words_nostops, dictionary=dico, coherence='c_v')
        coherence_values.append(coherence_model_lda.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dico, corpus=corpus, texts=data_words_nostops, start=1, limit=51, step=1)

C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\gensim\topic_coherence\text_analysis.py:448: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("stats accumulation interrupted; <= %d documents processed", self._num_docs)


In [ ]:
coherence_values

In [11]:
generate_lda = True
#generate_lda = False

# generate LDA model
import logging

if generate_lda :
    print("generate new LDA model")
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)
    ldamodel = LdaModel( corpus, 
                         num_topics = 40, 
                         id2word = dico,
                         passes = 2, 
                         random_state = 100, 
                         per_word_topics = True)
    
print(ldamodel)

2023-01-19 11:59:07,417 : INFO : using symmetric alpha at 0.025
2023-01-19 11:59:07,417 : INFO : using symmetric eta at 0.025
2023-01-19 11:59:07,433 : INFO : using serial LDA version on this node


generate new LDA model


2023-01-19 11:59:07,731 : INFO : running online (multi-pass) LDA training, 40 topics, 2 passes over the supplied corpus of 110895 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2023-01-19 11:59:07,731 : INFO : PROGRESS: pass 0, at document #2000/110895
2023-01-19 11:59:09,103 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-19 11:59:09,314 : INFO : topic #7 (0.025): 0.004*"jean" + 0.004*"lire" + 0.004*"ai" + 0.003*"entre" + 0.003*"celui" + 0.003*"claude" + 0.002*"lecture" + 0.002*"monde" + 0.002*"texte" + 0.002*"aime"
2023-01-19 11:59:09,314 : INFO : topic #13 (0.025): 0.005*"ai" + 0.005*"entre" + 0.004*"vie" + 0.004*"ont" + 0.004*"homme" + 0.004*"quelques" + 0.003*"celui" + 0.003*"jour" + 0.002*"moi" + 0.002*"plusieurs"
2023-01-19 11:59:09,314 : INFO : topic #35 (0.025): 0.005*"ai" + 0.003*"vie" + 0.003*"entre" + 0.003*"corps" + 0.003*"quelques

2023-01-19 11:59:21,720 : INFO : topic #35 (0.025): 0.025*"thanh" + 0.008*"ai" + 0.007*"han" + 0.006*"maries" + 0.005*"petrole" + 0.005*"dine" + 0.005*"vie" + 0.005*"miss" + 0.005*"toi" + 0.004*"uber"
2023-01-19 11:59:21,720 : INFO : topic #22 (0.025): 0.025*"usa" + 0.015*"france" + 0.011*"youtube" + 0.009*"president" + 0.008*"iran" + 0.007*"ans" + 0.007*"langue" + 0.007*"pays" + 0.006*"francais" + 0.005*"vie"
2023-01-19 11:59:21,720 : INFO : topic #25 (0.025): 0.014*"ai" + 0.013*"songs" + 0.012*"dien" + 0.008*"papa" + 0.008*"maman" + 0.007*"fils" + 0.006*"pardon" + 0.005*"chez" + 0.005*"vie" + 0.004*"vu"
2023-01-19 11:59:21,720 : INFO : topic #32 (0.025): 0.055*"ai" + 0.024*"viets" + 0.007*"viet" + 0.007*"vos" + 0.006*"idees" + 0.006*"toi" + 0.005*"nos" + 0.005*"gross" + 0.005*"aime" + 0.005*"grand"
2023-01-19 11:59:21,736 : INFO : topic diff=0.441659, rho=0.377964
2023-01-19 11:59:21,736 : INFO : PROGRESS: pass 0, at document #16000/110895
2023-01-19 11:59:23,167 : INFO : merging cha

2023-01-19 11:59:34,683 : INFO : topic #34 (0.025): 0.076*"blogspot" + 0.026*"ch" + 0.021*"feuilleton" + 0.014*"time" + 0.013*"good" + 0.012*"cut" + 0.012*"final" + 0.011*"vavar" + 0.010*"quin" + 0.009*"ivar"
2023-01-19 11:59:34,698 : INFO : topic diff=0.381823, rho=0.277350
2023-01-19 11:59:34,698 : INFO : PROGRESS: pass 0, at document #28000/110895
2023-01-19 11:59:37,064 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-19 11:59:37,237 : INFO : topic #3 (0.025): 0.028*"eme" + 0.024*"te" + 0.016*"es" + 0.014*"coeur" + 0.013*"toi" + 0.011*"ai" + 0.010*"vaille" + 0.009*"verdure" + 0.006*"contrainte" + 0.006*"fauteuils"
2023-01-19 11:59:37,237 : INFO : topic #1 (0.025): 0.007*"toulon" + 0.006*"maman" + 0.005*"fils" + 0.005*"debats" + 0.005*"restauration" + 0.004*"pays" + 0.004*"vie" + 0.004*"viet" + 0.004*"ont" + 0.004*"famille"
2023-01-19 11:59:37,252 : INFO : topic #6 (0.025): 0.014*"cloitre" + 0.006*"ai" + 0.005*"photo" + 0.005*"vases" + 0.004*"gra

...

2023-01-19 12:01:49,123 : INFO : topic #17 (0.025): 0.277*"people" + 0.060*"school" + 0.040*"allies" + 0.031*"da" + 0.018*"shot" + 0.016*"dien" + 0.013*"patricia" + 0.013*"candidates" + 0.013*"trimestre" + 0.012*"ana"
2023-01-19 12:01:49,123 : INFO : topic #2 (0.025): 0.028*"nos" + 0.012*"monde" + 0.007*"toute" + 0.007*"toutes" + 0.007*"sommes" + 0.006*"building" + 0.006*"entre" + 0.006*"pensee" + 0.005*"avons" + 0.005*"contre"
2023-01-19 12:01:49,123 : INFO : topic #20 (0.025): 0.022*"eau" + 0.013*"soleil" + 0.009*"table" + 0.009*"petit" + 0.007*"bois" + 0.007*"matin" + 0.006*"cafe" + 0.006*"jardin" + 0.005*"verre" + 0.004*"tete"
2023-01-19 12:01:49,123 : INFO : topic #25 (0.025): 0.015*"mere" + 0.012*"pere" + 0.012*"fille" + 0.009*"chez" + 0.009*"petit" + 0.008*"ans" + 0.007*"enfants" + 0.007*"petite" + 0.006*"maman" + 0.006*"fils"
2023-01-19 12:01:49,138 : INFO : topic #39 (0.025): 0.120*"color" + 0.056*"el" + 0.027*"del" + 0.023*"las" + 0.023*"mi" + 0.023*"los" + 0.022*"claro" + 0.

2023-01-19 12:01:58,580 : INFO : topic #19 (0.025): 0.058*"real" + 0.053*"back" + 0.046*"ve" + 0.021*"george" + 0.021*"evan" + 0.020*"word" + 0.020*"johnson" + 0.018*"street" + 0.015*"cut" + 0.013*"december"
2023-01-19 12:01:58,592 : INFO : topic #2 (0.025): 0.029*"nos" + 0.014*"monde" + 0.008*"toute" + 0.007*"toutes" + 0.007*"sommes" + 0.006*"entre" + 0.006*"avons" + 0.006*"pensee" + 0.006*"building" + 0.005*"nature"
2023-01-19 12:01:58,592 : INFO : topic #25 (0.025): 0.013*"mere" + 0.012*"pere" + 0.010*"fille" + 0.010*"ans" + 0.009*"chez" + 0.008*"petit" + 0.007*"enfants" + 0.007*"aime" + 0.007*"maison" + 0.006*"petite"
2023-01-19 12:01:58,608 : INFO : topic diff=0.085303, rho=0.131936
2023-01-19 12:02:00,292 : INFO : -13.679 per-word bound, 13111.8 perplexity estimate based on a held-out corpus of 2000 documents with 167921 words
2023-01-19 12:02:00,292 : INFO : PROGRESS: pass 1, at document #80000/110895
2023-01-19 12:02:01,533 : INFO : merging changes from 2000 documents into a mo

2023-01-19 12:02:08,018 : INFO : topic #39 (0.025): 0.055*"color" + 0.041*"el" + 0.024*"mi" + 0.023*"annie" + 0.021*"los" + 0.021*"yo" + 0.020*"las" + 0.020*"del" + 0.018*"ha" + 0.015*"al"
2023-01-19 12:02:08,018 : INFO : topic #18 (0.025): 0.020*"james" + 0.020*"lou" + 0.019*"anna" + 0.018*"brown" + 0.018*"tom" + 0.016*"park" + 0.016*"maria" + 0.012*"sessions" + 0.012*"toussaint" + 0.011*"taylor"
2023-01-19 12:02:08,033 : INFO : topic diff=0.096493, rho=0.131936
2023-01-19 12:02:08,033 : INFO : PROGRESS: pass 1, at document #92000/110895
2023-01-19 12:02:08,596 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-19 12:02:08,776 : INFO : topic #20 (0.025): 0.021*"eau" + 0.012*"soleil" + 0.010*"petit" + 0.009*"matin" + 0.008*"table" + 0.008*"jardin" + 0.008*"bois" + 0.006*"cafe" + 0.005*"verre" + 0.005*"herbe"
2023-01-19 12:02:08,776 : INFO : topic #33 (0.025): 0.020*"roman" + 0.011*"histoire" + 0.011*"texte" + 0.010*"livre" + 0.010*"lecture" + 0.009*"l

2023-01-19 12:02:17,370 : INFO : topic diff=0.050547, rho=0.131936
2023-01-19 12:02:17,386 : INFO : PROGRESS: pass 1, at document #104000/110895
2023-01-19 12:02:19,012 : INFO : merging changes from 2000 documents into a model of 110895 documents
2023-01-19 12:02:19,183 : INFO : topic #7 (0.025): 0.051*"jean" + 0.019*"marie" + 0.019*"merci" + 0.018*"pierre" + 0.015*"francois" + 0.014*"laurent" + 0.014*"franck" + 0.013*"vendredi" + 0.013*"claude" + 0.012*"lecture"
2023-01-19 12:02:19,183 : INFO : topic #31 (0.025): 0.060*"vie" + 0.057*"femmes" + 0.035*"hommes" + 0.023*"apprecierez" + 0.022*"femme" + 0.017*"celui" + 0.011*"filles" + 0.011*"blogs" + 0.008*"jeunes" + 0.006*"ont"
2023-01-19 12:02:19,183 : INFO : topic #28 (0.025): 0.019*"nos" + 0.011*"feu" + 0.011*"mort" + 0.011*"sang" + 0.010*"terre" + 0.010*"ont" + 0.008*"morts" + 0.007*"dieu" + 0.006*"amour" + 0.006*"hommes"
2023-01-19 12:02:19,183 : INFO : topic #0 (0.025): 0.008*"ont" + 0.006*"sera" + 0.005*"semaine" + 0.005*"quelques"

LdaModel(num_terms=100000, num_topics=40, decay=0.5, chunksize=2000)


In [15]:
ntopics = 40
ldamodel.show_topics(num_topics=ntopics,formatted=False)

[(0,
  [('poste', 0.016188696),
   ('ont', 0.007911111),
   ('commentaire', 0.0074634883),
   ('sera', 0.005317951),
   ('quelques', 0.0050784666),
   ('mois', 0.0043918234),
   ('semaine', 0.004177949),
   ('bonne', 0.00401126),
   ('jusqu', 0.003630745),
   ('fin', 0.0035813446)]),
 (1,
  [('jose', 0.019386519),
   ('jesus', 0.016284112),
   ('progres', 0.013145095),
   ('mme', 0.011679598),
   ('espagnol', 0.011105619),
   ('espagne', 0.010091849),
   ('juan', 0.008997664),
   ('luis', 0.008875348),
   ('bresil', 0.0067454195),
   ('portugais', 0.006617335)]),
 (2,
  [('nos', 0.025183218),
   ('monde', 0.014724522),
   ('sommes', 0.0079346355),
   ('toutes', 0.007818849),
   ('toute', 0.007692025),
   ('entre', 0.0071874214),
   ('pensee', 0.0059147244),
   ('avons', 0.0055627213),
   ('contre', 0.005540815),
   ('propre', 0.004640122)]),
 (3,
  [('te', 0.2399126),
   ('toi', 0.16567215),
   ('es', 0.11786385),
   ('dis', 0.04148416),
   ('veux', 0.028170463),
   ('sais', 0.02800761

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare( ldamodel,
                                      gensim.matutils.corpus2csc(corpus),
                                      dictionary = ldamodel.id2word)

pyLDAvis.display(vis)

C:\Users\brune\anaconda3\envs\CoursHN\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [17]:
ldc = ldamodel[corpus]

Tout d'abord, on souhaite un tableau qui liste la thématique majoritaire pour chaque document, accompagnée par ses mots les plus probables.

In [18]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    i=0
    for i, row in enumerate(ldamodel[corpus]):
        #print(row[0])            
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        #print(row)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, texts=doc_set)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

C:\Users\brune\AppData\Local\Temp\ipykernel_3408\4166790178.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,8,0.3358,"corps, yeux, nuit, voix, tete, main, homme, mo...",le premier jour du premier mois de l'année res...
1,1,0,0.0250,"poste, ont, commentaire, sera, quelques, mois,...",
2,2,8,0.2729,"corps, yeux, nuit, voix, tete, main, homme, mo...",âmes perduescauses perduesun constat s'impose ...
3,3,8,0.3062,"corps, yeux, nuit, voix, tete, main, homme, mo...",pour y croîtrej'ai enterré quelque chose de mo...
4,4,0,0.0250,"poste, ont, commentaire, sera, quelques, mois,...",
5,5,27,0.2141,"ciel, mer, vent, ville, jour, lumiere, terre, ...",des sentiments le poursuivent qui lui cuisent ...
6,6,5,0.3723,"amour, monde, beaute, entre, desir, ame, toute...",y a des journées parfaites et fébriles où j'ar...
7,7,21,0.6079,"moi, vie, monde, mal, choses, personne, mieux,...",je ne suis ni normand ni féministe ni chanteur...
8,8,21,0.3723,"moi, vie, monde, mal, choses, personne, mieux,...",je ne sais pas ce qui me peine et je sais à p...
9,9,8,0.3470,"corps, yeux, nuit, voix, tete, main, homme, mo...",sans les méchants les saints ne le sont guère ...


On peut vouloir obtenir les documents les plus "représentatifs" de chaque thématique (attention, au sens de p(z|d)).

In [19]:
sent_topics_sorteddf = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf = pd.concat([sent_topics_sorteddf, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.9190,"poste, ont, commentaire, sera, quelques, mois,...","Je n'oublie pas le 93 (visite d'Epinay demain,..."
1,1,0.7562,"jose, jesus, progres, mme, espagnol, espagne, ...",Amphibian shit.Prêtres de la mort.
2,2,0.9817,"nos, monde, sommes, toutes, toute, entre, pens...",nos pas ne (sont pas) nos pas ne (sont pas) n...
3,3,0.8124,"te, toi, es, dis, veux, sais, fais, vois, atte...",j'attends j'attends j'attends j'attends j'atte...
4,4,0.8607,"saint, thomas, louis, jean, paris, paul, jacqu...","Hans Holbein le Jeune (mort le 29/11/1543), Le..."
5,5,0.8422,"amour, monde, beaute, entre, desir, ame, toute...",L'amour ne rend pasles amoureux aveuglesc'est ...
6,6,0.7559,"pascale, station, metro, chateau, berri, uqam,...",retour dans la brumephares léchant la garennec...
7,7,0.9025,"commentaire, jean, marie, merci, dominique, fr...",© photo : Fred SteinUne chronique d'Emmanuelle...
8,8,0.9961,"corps, yeux, nuit, voix, tete, main, homme, mo...",j'ouvre les yeux / mon rêve m'échappe / j'ouvr...
9,9,0.9625,"theatre, scene, saison, spectacle, piece, musi...",ta voix est un lieuta voix est un espaceta voi...


In [20]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts

0     19107
8     17354
27    14859
21     8604
25     8518
33     6570
12     4470
32     3868
28     3714
14     3231
20     2808
30     2264
5      1811
22     1641
7      1617
9      1330
37     1249
4      1078
2       946
38      882
29      880
24      772
10      535
23      434
39      278
31      229
3       216
16      181
34      173
36      167
17      157
13      149
11      147
1       111
6       107
19       94
18       92
26       86
35       85
15       81
Name: Dominant_Topic, dtype: int64